In [54]:
import re
import pandas as pd
import numpy as np
import gspread
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [3]:
gc = gspread.service_account(filename='/home/gbotemi/Documents/gbotemi_bolarinwa/gspread-project/secret.json') # use your google service file account here
file = "https://docs.google.com/spreadsheets/d/1SUlcukpgxf6pfFQbj6DKQZXSlOhyBOSuBdbmQ2ZbhTM/edit?usp=sharing"

sh = gc.open_by_url(file)
worksheet = sh.get_worksheet(1)

In [161]:
data = pd.DataFrame(worksheet.get_all_records())

In [154]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   S#                       496 non-null    int64 
 1   Date                     496 non-null    object
 2   Islamic Date             496 non-null    object
 3   Blast Day Type           496 non-null    object
 4   Holiday Type             496 non-null    object
 5   Time                     496 non-null    object
 6   City                     496 non-null    object
 7   Latitude                 496 non-null    object
 8   Longitude                496 non-null    object
 9   Province                 496 non-null    object
 10  Location                 496 non-null    object
 11  Location Category        496 non-null    object
 12  Location Sensitivity     496 non-null    object
 13  Open/Closed Space        496 non-null    object
 14  Influencing Event/Event  496 non-null    o

In [231]:
data.head()

,s#,date,islamic date,blast day type,holiday type,time,city,latitude,longitude,province,location,location category,location sensitivity,open/closed space,influencing event/event,target type,targeted sect if any,killed min,killed max,injured min,injured max,no. of suicide blasts,explosive weight (max),hospital names,temperature(c),temperature(f)
0,1,1995-11-19,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaT,Islamabad,33.7180,73.0718,Capital,Egyptian Embassy,Foreign,High,Closed,NaN,Foreigner,None,14.0,15.0,NaN,60.0,2.0,NaN,NaN,15.835,60.503
1,2,2000-11-06,10 SHa`baan 1421 A.H,Working Day,NaN,NaT,Karachi,24.9918,66.9911,Sindh,office of Nawa-e-Waqt,Office Building,Low,Closed,NaN,Media,None,NaN,3.0,NaN,3.0,1.0,NaN,NaN,23.770,74.786
2,3,2002-05-08,25 safar 1423 A.H,Working Day,NaN,07:45:00,Karachi,24.9918,66.9911,Sindh,Pakistan Navy bus Parked outside Five Star She...,Hotel,Medium,Closed,NaN,Foreigner,Christian,13.0,15.0,20.0,40.0,1.0,25.0,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,2002-06-14,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00,Karachi,24.9918,66.9911,Sindh,US Consulate Civil Lines Area,Foreign,High,Closed,NaN,Foreigner,Christian,NaN,12.0,NaN,51.0,1.0,NaN,NaN,31.430,88.574
4,5,2003-07-04,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaT,Quetta,30.2095,67.0182,Baluchistan,Imambargah MeCongy Road Quetta,Religious,Medium,Closed,during Friday prayer,Religious,Shiite,44.0,47.0,NaN,65.0,1.0,NaN,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616


In [162]:
data = data.replace("", np.nan)
data = data.replace("N/A", np.nan)
data = data.replace("NA", np.nan)
data.columns = data.columns.str.lower() # lower columns values

In [163]:
data['date'] = pd.to_datetime(data['date'], errors='coerce')	
data['time'] = pd.to_datetime(data['time'], errors='coerce').dt.time

In [164]:
data["explosive weight (max)"] = data["explosive weight (max)"].replace(r"to", "-", regex=True)
data["explosive weight (max)"] = data["explosive weight (max)"].str.replace(r"[^0-9-]+", "", regex=True)
data["explosive weight (max)"] = data["explosive weight (max)"].replace(r"\d*-", "", regex=True).replace("", np.nan).astype("float")

In [227]:
data["injured max"] = data["injured max"].replace(r"[^A-z ]", "", regex=True)
data["injured max"] = data["injured max"].replace(r"[A-z]", "", regex=True)
data["injured max"] = data["injured max"].replace("", np.nan).astype("float")

In [243]:
data.columns

Index(['s#', 'date', 'islamic date', 'blast day type', 'holiday type', 'time',
       'city', 'latitude', 'longitude', 'province', 'location',
       'location category', 'location sensitivity', 'open/closed space',
       'influencing event/event', 'target type', 'targeted sect if any',
       'killed min', 'killed max', 'injured min', 'injured max',
       'no. of suicide blasts', 'explosive weight (max)', 'hospital names',
       'temperature(c)', 'temperature(f)'],
      dtype='object')